In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

In [2]:
torch.set_printoptions(precision=10)

In [3]:
plikembd = open("../embeddings_small.txt")

embeddings = {}

embeddings['🈳'] = np.zeros(400)

embeddingscuda_chars = {}
embeddingscuda_chars['🈳'] = 0

embeddingscuda = torch.tensor([]).to("cuda:0")
embeddingscuda =  torch.cat((embeddingscuda, torch.zeros((400)).to("cuda:0")), 0)
#mbeddingscuda = torch.cat((embeddingscuda, torch.zeros((1, 400), dtype=torch.float64).to("cuda:0")), 0)

print(embeddingscuda.shape)

for cnt, line in enumerate(plikembd.readlines()):
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    embedding = torch.tensor([float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]).to("cuda:0")
    
    embeddingscuda_chars[znak] = cnt+1
    embeddingscuda = torch.cat((embeddingscuda, embedding), 0)
    #embeddingscuda[cnt+1] = embedding

    
    embeddings[znak] = embedding

#embeddingscuda = embeddingscuda/10
    


embeddingscuda = embeddingscuda.reshape(len(embeddings), -1)


#print(embeddings['a'])
#print(embeddingscuda_chars['a'])
print(embeddingscuda.shape)
print(embeddingscuda[embeddingscuda_chars['a']])
#-0.18429723

torch.Size([400])
torch.Size([82, 400])
tensor([-0.1842972338,  0.1992288828, -0.1093217134,  0.0635560006,
        -0.0170881711, -0.0069951965,  0.1790032834, -0.1581476331,
         0.0322009474, -0.1496459693, -0.1595581025, -0.2537648678,
         0.2232226729, -0.1215698719,  0.0059251408, -0.1644940972,
         0.0264174044,  0.0696226805, -0.0749481246,  0.2048900872,
        -0.0481871516, -0.1348041445,  0.1884992272, -0.0314494744,
         0.1081886366,  0.0045932876, -0.1269595921,  0.1936095357,
        -0.0679676980,  0.1462467760, -0.2293221653, -0.2348391116,
        -0.1658135504, -0.0536651835,  0.0345649682, -0.0555758253,
         0.1800168753, -0.1355556846,  0.0986127034, -0.0864410177,
        -0.1333196312, -0.0306926053,  0.2023293376,  0.1658294201,
         0.1575038880,  0.1013869643, -0.0837135166, -0.0609861165,
        -0.1243505403, -0.0618062057, -0.0087015610, -0.1109482422,
        -0.1424127370,  0.1705076247, -0.0412648059, -0.1866530329,
        

In [4]:
polskie = open("../100k.txt", encoding='utf8')
slowa = [slowo.replace("\n", "") for slowo in polskie.readlines()]
polskie.close()
print(slowa[:100])
print(len(slowa))
slowa = slowa[:10000]

['abmicro', '527579', 'agro2011', 'nakamichi', 'dugant', '29082009', 'hannah1', 'ma12po', '580813', 'lukasz4', 'kriket', 'werohaze', 'dron14121987', 'pinot1', 'maria59', 'moltrex', 'kilim', 'Zabeczka', '7937', '7maluszek', 'gacek16', 'ruleta1', '17Cyprian', 'maj2006', 'o2skarek', 'anno01', 'gusciol', 'gumis1', 'malaz11', 'watomzon', 'rumianek', 'neoisp', 'zsofie', 'zipera', '141084', 'ultraboy', '110567', 'anbog', 'troyd34', 'roxboro', 'werunia', 'misiokrzysio', 'hydra6', 'grzes10', 'kakigewy', 'filip98', '20021970', 'natka777', 'patricia', 'polisnova', 'miko20', 'prj2501', 'grobian1', '1972ml', 'robo21101967', 'jw17061958', 'gregordj', 'insko2000', 'maclear', 'zl060374', 'asia7', 'misiu777', 'varius2', 'lgflatron', 'rele1', '130178', 'centrala', 'filoman', '41karat2', '84jaysinner', 'agatina', 'ambasada', 'maly17', 'goszkam', '1branka', 'dasti', 'petro111', '070165', '20111965', 'triforce', 'bogumil1', 'anna3005', 'dron10011001', 'santaclara', 'pas2526', '11011975', 'rest12', 'j87neo'

In [5]:
def podobienstwo_cosinusowe(wektor1, wektor2, podobienstwo_min=0.01):
    if torch.sum(wektor1) == 0 or torch.sum(wektor2) == 0:
        return 0
    
    sumaup = torch.sum(wektor1 * wektor2)
    s1 = torch.sum(wektor1**2)
    s2 = torch.sum(wektor2**2)

    outt = sumaup/(torch.sqrt(s1)*torch.sqrt(s2))
    
    if outt>=podobienstwo_min:
        return outt
    else:
        return 0 
podobienstwo_cosinusowe(embeddings['a'], embeddings['e'])

tensor(0.2830401361, device='cuda:0')

In [8]:
def znajdz_najblizszy(wektor, embeddings):
    najblizszy = '🈳'
    podobienstwo = 0
    
    if torch.sum(wektor) == 0:
        return najblizszy, 1
    
    for key in embeddings:
        pdb = podobienstwo_cosinusowe(wektor, embeddings[key])
        #przemnozyc macierze na raz, bez for'a
        if pdb > podobienstwo:
            podobienstwo = pdb
            najblizszy = key
    return najblizszy, podobienstwo

print(str(datetime.now()))
znajdz_najblizszy(torch.randn((400)).to("cuda:0"), embeddings)
print(str(datetime.now()))

2019-07-14 19:34:53.074755


TypeError: sum(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [9]:
# print(str(datetime.now()))
# sumaup = torch.sum(embeddingscuda[embeddingscuda_chars['a']] * embeddingscuda, dim=1, dtype=torch.float64).to("cuda:0")



# s1 = torch.sum(embeddingscuda[embeddingscuda_chars['a']]**2, dtype=torch.float64).to("cuda:0")

# s2 = torch.sum(embeddingscuda**2, dim=1, dtype=torch.float64).to("cuda:0")

# wynik = sumaup/(torch.sqrt(s1)*torch.sqrt(s2))

# wynik[0] = 0

# print(str(datetime.now()))
# print(wynik)
# print(torch.max(wynik))
# print(list(embeddingscuda_chars.keys())[torch.argmax(wynik).item()])

In [10]:
# embeddingscuda[embeddingscuda_chars['a']]

In [11]:
podobienstwo_cosinusowe(embeddings['a'], embeddings['e'])

tensor(0.2830401361, device='cuda:0')

In [12]:
longestword = 0

unikalneznaki = []
for slowo in slowa:
    for literka in slowo:
        if literka not in unikalneznaki:
            unikalneznaki.append(literka)
    if len(slowo) > longestword:
        longestword = len(slowo)
            
print(unikalneznaki)
print(longestword)
print(len(unikalneznaki))

['a', 'b', 'm', 'i', 'c', 'r', 'o', '5', '2', '7', '9', 'g', '0', '1', 'n', 'k', 'h', 'd', 'u', 't', '8', 'p', '3', 'l', 's', 'z', '4', 'e', 'w', 'x', 'Z', '6', 'C', 'y', 'j', 'f', 'v', 'F', 'P', 'R', 'O', 'T', 'A', 'X', 'L', 'E', 'S', 'I', 'K', 'N', 'D', 'M', 'q', 'H', 'U', 'J', 'Q', 'G', 'W', 'B', 'V', 'Y', '@', '.', '#', '!', ';', '*']
15
68


In [13]:
print(list(embeddings.keys()))
print(len(list(embeddings.keys())))

['🈳', ' ', '!', '#', '$', '%', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
82


In [14]:
print(len(embeddings[list(embeddings.keys())[0]]))

400


In [15]:
onehotvectors = {}
klasyid = {}

for cnt, litera in enumerate(list(embeddings.keys())):
    vector = [0 for __ in range(len(embeddings.keys()))]
    vector[cnt] = 1
    onehotvectors[litera] = torch.Tensor(vector).to("cuda:0")
    
    klasyid[litera] = cnt
    
print(len(onehotvectors))
print(len(onehotvectors['🈳']))
print(len(klasyid))
print(klasyid)

82
82
82
{'🈳': 0, ' ': 1, '!': 2, '#': 3, '$': 4, '%': 5, '*': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ';': 22, '=': 23, '@': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '^': 53, '_': 54, '`': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81}


In [16]:
print(onehotvectors['🈳'])
print(len(onehotvectors['🈳']))

dlugoschotvectora = len(onehotvectors['🈳'])

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
82


In [17]:
print("ilosc wejsc dyskryminatora: " + str(dlugoschotvectora * longestword))
print("ilosc wyjsc generatora: " + str(len(embeddings['🈳']) * longestword))

ilosc wejsc dyskryminatora: 1230
ilosc wyjsc generatora: 6000


In [18]:
dataset_real = []
iloscslow = 0

pusty = torch.zeros(dlugoschotvectora).to("cuda:0")

for cnt, slowo in enumerate(slowa):
    slowoencoded = []
    
    for index in range(longestword):
        if index < len(slowo):
            slowoencoded.append(onehotvectors[slowo[index]])
        else:
            slowoencoded.append(pusty)
        iloscslow = iloscslow + 1
    dataset_real = dataset_real + slowoencoded

print(len(dataset_real))

150000


In [19]:
dataset_real = torch.Tensor([]).to("cuda:0")
iloscslow = 0

#pusty = [0 for __ in range(len(embeddings.keys()))]
pusty = torch.zeros(len(embeddings.keys())).to("cuda:0")

for cnt, slowo in tqdm(enumerate(slowa)):
    for index in range(longestword):
        if index<len(slowo):
            #slowoencoded = slowoencoded + onehotvectors[slowo[index]]
            dataset_real = torch.cat((dataset_real, onehotvectors[slowo[index]]), 0)
            # slowoencoded.append(slowo[index])
        else:
            #slowoencoded = slowoencoded + [0 for __ in range(len(unikalneznaki))]
            dataset_real = torch.cat((dataset_real, pusty), 0)
    #torch.cat((slowoencoded, pusty), 0)
    #dataset_real = np.append(dataset_real, slowoencoded)
    iloscslow = iloscslow + 1
    #if len(slowo) < longestword:
    #    for __ in range(longestword - len(slowo)):
    #        #dataset_real.append("XD")
    #        slowoencoded = [slowoencoded[-1]] + slowoencoded[:-1]
    #        dataset_real = dataset_real + slowoencoded
    #        iloscslow = iloscslow + 1
    
#dataset_real = np.array(dataset_real)

dataset_real = dataset_real.reshape(iloscslow, -1)

print(dataset_real.shape)

10000it [01:04, 153.98it/s]


torch.Size([10000, 1230])


## TODOs
Zrobić FeatureLoss'a

In [20]:
vectors = [
           [embeddingscuda[embeddingscuda_chars['a']], 
           embeddingscuda[embeddingscuda_chars['b']], 
           embeddingscuda[embeddingscuda_chars['c']], 
           embeddingscuda[embeddingscuda_chars['d']], 
           embeddingscuda[embeddingscuda_chars['e']],
           embeddingscuda[embeddingscuda_chars['f']],
           embeddingscuda[embeddingscuda_chars['g']],
           embeddingscuda[embeddingscuda_chars['h']],
           [embeddingscuda[embeddingscuda_chars['a']], 
           embeddingscuda[embeddingscuda_chars['b']], 
           embeddingscuda[embeddingscuda_chars['c']], 
           embeddingscuda[embeddingscuda_chars['d']], 
           embeddingscuda[embeddingscuda_chars['e']],
           embeddingscuda[embeddingscuda_chars['f']],
           embeddingscuda[embeddingscuda_chars['g']],
           embeddingscuda[embeddingscuda_chars['h']],
           [embeddingscuda[embeddingscuda_chars['a']], 
           embeddingscuda[embeddingscuda_chars['b']], 
           embeddingscuda[embeddingscuda_chars['c']], 
           embeddingscuda[embeddingscuda_chars['d']], 
           embeddingscuda[embeddingscuda_chars['e']],
           embeddingscuda[embeddingscuda_chars['f']],
           embeddingscuda[embeddingscuda_chars['g']],
           embeddingscuda[embeddingscuda_chars['h']],
           [embeddingscuda[embeddingscuda_chars['a']], 
           embeddingscuda[embeddingscuda_chars['b']], 
           embeddingscuda[embeddingscuda_chars['c']], 
           embeddingscuda[embeddingscuda_chars['d']], 
           embeddingscuda[embeddingscuda_chars['e']],
           embeddingscuda[embeddingscuda_chars['f']],
           embeddingscuda[embeddingscuda_chars['g']],
           embeddingscuda[embeddingscuda_chars['h']],
           [embeddingscuda[embeddingscuda_chars['a']], 
           embeddingscuda[embeddingscuda_chars['b']], 
           embeddingscuda[embeddingscuda_chars['c']], 
           embeddingscuda[embeddingscuda_chars['d']], 
           embeddingscuda[embeddingscuda_chars['e']],
           embeddingscuda[embeddingscuda_chars['f']],
           embeddingscuda[embeddingscuda_chars['g']],
           embeddingscuda[embeddingscuda_chars['h']],
           [embeddingscuda[embeddingscuda_chars['a']], 
           embeddingscuda[embeddingscuda_chars['b']], 
           embeddingscuda[embeddingscuda_chars['c']], 
           embeddingscuda[embeddingscuda_chars['d']], 
           embeddingscuda[embeddingscuda_chars['e']],
           embeddingscuda[embeddingscuda_chars['f']],
           embeddingscuda[embeddingscuda_chars['g']],
           embeddingscuda[embeddingscuda_chars['h']]]
]
def cuda(vectors):
    #strings = []
    wyniki_loss = []
    onehots = torch.tensor([]).to("cuda:0")
    
    s2 = torch.sum(embeddingscuda**2, dim=1, dtype=torch.float64).to("cuda:0")
    
    for cnt in tqdm(range(len(vectors))):

        #string = ""

        for single in vectors[cnt]:
            
            sumaup = torch.sum(single * embeddingscuda, dim=1, dtype=torch.float64).to("cuda:0")
            s1 = torch.sum(single**2, dtype=torch.float64).to("cuda:0") 

            wynik = sumaup/(torch.sqrt(s1)*torch.sqrt(s2))
            wynik[0] = 0
            #string=string+list(embeddingscuda_chars.keys())[torch.argmax(wynik).item()]
            wyniki_loss.append(torch.max(wynik).item())
            onehots = torch.cat((onehots, onehotvectors[list(embeddingscuda_chars.keys())[torch.argmax(wynik).item()]]), 0)
            
    #strings.append(string)
        
    return "", onehots.reshape(len(vectors), -1), np.mean(np.array(wyniki_loss))
%time cuda(vectors)

SyntaxError: invalid syntax (<ipython-input-20-9331869679b5>, line 51)

In [25]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()     
        
        self.fc_in = nn.Linear(1230, 800)
        self.fc_h1 = nn.Linear(800, 400)
#         self.fc_h2 = nn.Linear(400, 200)
#         self.fc_h3 = nn.Linear(200, 100)
#         self.fc_h4 = nn.Linear(100, 50)
        self.fc_h5 = nn.Linear(400, 20)
        self.fc_out = nn.Linear(20, 1)
        
    def forward(self, x):      
        x = self.fc_in(x)
        x = F.relu(x)
        
        x = self.fc_h1(x)
        x = F.relu(x)
        
#         x = self.fc_h2(x)
#         x = F.relu(x)
        
#         x = self.fc_h3(x)
#         x = F.relu(x)
        
#         x = self.fc_h4(x)
#         x = F.relu(x)
        
        x = self.fc_h5(x)
        x = F.relu(x)
        
        x = self.fc_out(x)
        x = F.sigmoid(x)
        
        return x

In [26]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.fc_in = nn.Linear(500, 800)
        self.fc_h1 = nn.Linear(800, 3500)
#         self.fc_h2 = nn.Linear(1200, 2000)
#         self.fc_h3 = nn.Linear(2000, 2800)
#         self.fc_h4 = nn.Linear(2800, 3500)
        self.fc_out = nn.Linear(3500, 6000)
        
        self.activation1 = nn.PReLU()
        
    def forward(self, x):
        x = self.fc_in (x)
        x = F.relu(x)
        
        x = self.fc_h1(x)
        x = F.relu(x)
        
#         x = self.fc_h2(x)
#         x = F.relu(x)
        
#         x = self.fc_h3(x)
#         x = F.relu(x)
        
#         x = self.fc_h4(x)
#         x = F.relu(x)
        
        x = self.fc_out(x)
        x = self.activation1(x)
        
        return x

In [27]:
#dataset_real = np.array(dataset_real).reshape(iloscslow, -1)

In [28]:
#print(dataset_real.shape)

In [29]:
print(dataset_real[1])

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')


In [30]:
discriminator = Discriminator().to("cuda:0")
generator = Generator().to("cuda:0")

TensorReal = dataset_real

TensorFakeOuts = torch.zeros(len(dataset_real)).to("cuda:0")
TensorRealOuts = torch.ones(len(dataset_real)).to("cuda:0")

TensorOuts = torch.cat((TensorFakeOuts, TensorRealOuts), 0).to("cuda:0")

In [31]:
TensorReal.shape

torch.Size([10000, 1230])

In [32]:
criterion = nn.BCELoss()
criterion_segment = nn.CrossEntropyLoss()

In [33]:
def translate_outputs_to_onehots(npoutputs, dlugoscunikalne, embeddings, longestword, onehots):
    zwrot = []
    
    skok = int(len(npoutputs) / multiprocessing.cpu_count())
    ilezostalo = len(npoutputs) % multiprocessing.cpu_count()
    
    padding = 0
    if skok == 1:
        padding = 1
    
    podzial = [(x*skok,x*skok+skok) if (x*skok+skok-1-x*skok) > 0 else (x*skok,x*skok+padding) for x in range(multiprocessing.cpu_count())]
    
    if ilezostalo != 0:
        podzial[-1] = (podzial[-1][0], podzial[-1][1]+ilezostalo)

    #print([(i[0],i[1]) for i in podzial])
    
    results = []
    results = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(translate_outputs_to_onehots_parallel)(npoutputs[i[0]:i[1]], dlugoscunikalne, embeddings, onehots) for i in podzial)
    
    results2 = []
    results_onehots = []
    results_loss = []
    
    for cnt in tqdm(range(len(results))):
        results2 = results2 + results[cnt][0]
        results_onehots = results_onehots + results[cnt][1]
        results_loss = results_loss + results[cnt][2]
        
    loss = torch.mean(torch.Tensor(np.array(results_loss)).to("cuda:0").reshape(-1))
    
    return np.array(results2).reshape(-1, longestword), np.array(results_onehots).reshape(len(npoutputs), -1), loss.cpu().detach().numpy()

def translate_outputs_to_onehots_parallel(npoutputs, jump, embeddings, onehots):
    zwrot = []
    zwrotonehots = []
    zwrotloss = []
    
    for x in npoutputs:
        slowo = []
        onehot = []
        loss = []
        for i in range(int(len(x)/jump)):
            najblizszy = znajdz_najblizszy(x[i*jump:i*jump+jump-1], embeddings)
            slowo.append(najblizszy[0])
            onehot = onehot + onehots[najblizszy[0]]
            loss.append(najblizszy[1])
        
        zwrotonehots.append(onehot)
        zwrot.append(slowo)
        zwrotloss.append(loss)
        
    return zwrot, zwrotonehots, zwrotloss


def translate_outputs_to_onehots_on_cuda(npoutputs, dlugoscunikalne, embeddings, longestword, onehots):
    zwrot = []
    zwrotonehots = torch.Tensor([]).to("cuda:0")
    zwrotloss = []
    
    for cnt in tqdm(range(len(npoutputs))):
        x = npoutputs[cnt]
        slowo = []
        onehot = torch.Tensor([]).to("cuda:0")
        loss = []
        for i in range(int(len(x)/dlugoscunikalne)):
            print(str(datetime.now()))
            najblizszy = znajdz_najblizszy(x[i*dlugoscunikalne:i*dlugoscunikalne+dlugoscunikalne], embeddings)
            print(str(datetime.now()))
            slowo.append(najblizszy[0])
            onehot = torch.cat((onehot, onehots[najblizszy[0]]), 0)
            loss.append(najblizszy[1])
        zwrotonehots = torch.cat((zwrotonehots, onehot), 0)
        zwrot.append(slowo)
        zwrotloss.append(loss)
    zwrotonehots = zwrotonehots.reshape(len(npoutputs), -1)
    
    return zwrot, zwrotonehots, zwrotloss

In [34]:
TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")

In [32]:
xx = TensorFakeIns.cpu().detach().numpy()

In [33]:
xx.shape

(10000, 6000)

In [34]:
#print(xx[0])
#xx = xx[:1]
#with np.printoptions(threshold=np.inf):
#    print(xx)

In [56]:
translated1, translated2, translated_loss = cuda(TensorFakeIns.reshape(10000, -1))
print(translated_loss)


  0%|                                                                             | 1/10000 [00:07<20:11:00,  7.27s/it]

KeyboardInterrupt: 

In [33]:
TensorFakeTranslated = torch.Tensor(translated2).to("cuda:0")

In [34]:
TensorFakeTranslated.shape

torch.Size([100, 984])

In [35]:
discriminator(TensorFakeTranslated)

C:\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.4755],
        [0.4748],
        [0.4748],
        [0.4751],
        [0.4755],
        [0.4747],
        [0.4744],
        [0.4764],
        [0.4747],
        [0.4743],
        [0.4761],
        [0.4752],
        [0.4752],
        [0.4737],
        [0.4742],
        [0.4744],
        [0.4770],
        [0.4752],
        [0.4747],
        [0.4742],
        [0.4760],
        [0.4761],
        [0.4748],
        [0.4753],
        [0.4756],
        [0.4761],
        [0.4755],
        [0.4771],
        [0.4755],
        [0.4755],
        [0.4755],
        [0.4745],
        [0.4756],
        [0.4756],
        [0.4758],
        [0.4744],
        [0.4768],
        [0.4753],
        [0.4763],
        [0.4769],
        [0.4755],
        [0.4767],
        [0.4755],
        [0.4747],
        [0.4757],
        [0.4759],
        [0.4756],
        [0.4757],
        [0.4753],
        [0.4749],
        [0.4751],
        [0.4763],
        [0.4761],
        [0.4767],
        [0.4766],
        [0

In [24]:
optimizerGenerator = optim.Adam(generator.parameters(), lr=1e-3)
optimizerClassifier = optim.Adam(discriminator.parameters(), lr=1e-3)

In [25]:
for epoch in range(1000):
    TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
    TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")
    
    xx = TensorFakeIns.cpu().detach().numpy()
    translated1, translated2, featureloss = translate_outputs_to_onehots(xx, 400, embeddings, longestword, onehotvectors)
    TensorFakeTranslated = torch.Tensor(translated2).to("cuda:0")
    
    print(translated1[0])
    
    TensorReal = torch.Tensor(dataset_real).to("cuda:0")
    
    TensorIns = torch.cat((TensorFakeTranslated.detach(), TensorReal), 0).to("cuda:0")
    
    ## DISCRIMINATOR
    discriminator.train()
    optimizerClassifier.zero_grad()
    
    y_ = discriminator(TensorIns)
    lossDiscriminator = criterion(y_, TensorOuts)
    
    lossDiscriminator.backward()
    optimizerClassifier.step()
    
    ## GENERATOR
#     TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
#     TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")
    
#     xx = TensorFakeIns.cpu().detach().numpy()
#     translated1, translated2, featureloss = translate_outputs_to_onehots(xx, 400, embeddings, longestword, onehotvectors)
#     TensorFakeTranslated = torch.Tensor(translated2).to("cuda:0")
    
    
    generator.train()
    optimizerGenerator.zero_grad()
    
    y_ = discriminator(TensorFakeTranslated)
    lossGenerator = criterion(y_, TensorRealOuts) + (1-featureloss)*10
    
    lossGenerator.backward()
    optimizerGenerator.step()
    
    #print(xx)
    print(lossDiscriminator, lossGenerator)

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 4001.24it/s]


['$' 'D' 's' '-' 'x' 'U' 'q' 'w' 's' '.' 'L' 'I' 'D' '.' 'P']


C:\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Anaconda3\lib\site-packages\torch\nn\functional.py:2016: UserWarning: Using a target size (torch.Size([20000])) that is different to the input size (torch.Size([20000, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
C:\Anaconda3\lib\site-packages\torch\nn\functional.py:2016: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


tensor(0.6930, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(9.5639, device='cuda:0', grad_fn=<AddBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]


['c' 'V' '0' '=' 'Q' 'N' 'l' 'p' '=' '.' 'L' 'I' 'P' 'a' 'W']
tensor(0.6708, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(9.6079, device='cuda:0', grad_fn=<AddBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]


['w' 'm' 'D' '-' 'w' '/' 'b' 'n' '5' '-' 'W' 'O' '.' '.' '=']
tensor(0.6520, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) tensor(9.6475, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [ ]:
xx